In [3]:
import os
os.sys.path

# Installing libraries
!pip3 install opencv-python
!pip3 install numpy
!pip3 install pandas

# Importing

import numpy as np
import pandas as pd

In [4]:
# Reading Excel files
df = pd.read_excel(r"C:\Users\Keithon\Desktop\Capstone\Data\Test.xlsx",sheet_name='WorkstageRecordItem')
df1 = pd.read_excel(r"C:\Users\Keithon\Desktop\Capstone\Data\Test.xlsx",sheet_name='JobCost')
df2 = pd.read_excel(r"C:\Users\Keithon\Desktop\Capstone\Data\Test.xlsx",sheet_name='JobOrder')

df = df.drop(columns =["Id","JobOrderId","Status","Reference","TechnicianSignature_StampNo","ApproverRole","ReleaseToSignatorySignature_StampNo","TechnicianSignature_SignedOn","CompletedByUserId","CompletedByFullname","ApprovedByUserId","ApprovedByFullname","ApprovedOn","ReleaseToSignatorySignature_DigitalCertificateId","ReleaseToSignatorySignature_Signature","ReleaseToSignatorySignature_SignedByUserId","ReleaseToSignatorySignature_SignedOn","TechnicianSignature_DigitalCertificateId","TechnicianSignature_Signature","TechnicianSignature_SignedByUserId"], axis = 1)
df1 = df1.drop(["MWC"], axis = 1)
df2 = df2.drop(["MWC","ReviewerSignature_Signature","ComplianceDeclarationRecord_ReleaseToSignatorySignature_Signature","SignWorkscopeAndCRRBy","WorkScopeId","SignWorkscopeAndCRRBy","TechnicianSignature_StampNo","TechnicianSignature_SignedOn","TechnicianSignature_SignedByUserId","TechnicianSignature_Signature","TechnicianSignature_DigitalCertificateId","ReviewerSignature_StampNo","ReviewerSignature_SignedOn","ReviewerSignature_SignedByUserId","ReviewerSignature_DigitalCertificateId","ComplianceDeclarationRecord_TechnicianSignature_StampNo","ComplianceDeclarationRecord_ReleaseToSignatorySignature_StampNo","ComplianceDeclarationRecord_TechnicianSignature_SignedOn","ComplianceDeclarationRecord_TechnicianSignature_SignedByUserId","ComplianceDeclarationRecord_TechnicianSignature_Signature","ComplianceDeclarationRecord_TechnicianSignature_DigitalCertificateId","ComplianceDeclarationRecord_ReleaseToSignatorySignature_SignedOn","ComplianceDeclarationRecord_ReleaseToSignatorySignature_SignedByUserId","ComplianceDeclarationRecord_ReleaseToSignatorySignature_DigitalCertificateId","RowVersion","ComplianceDeclarationRecord_ReviewStatus","ComplianceDeclarationRecord_LastDeclaredOn","RejectReason","ComplianceDeclarationRecord_LastDeclaredByUserId","ComplianceDeclarationRecord_LastDeclaredByFullName","ComplianceDeclarationRecord_LastApprovedOn","ComplianceDeclarationRecord_LastApprovedByUserId","ComplianceDeclarationRecord_LastApprovedByFullName","ComplianceDeclarationRecord_IsDeclared","ReviewedBy","NSN","SignWorkscopeAndCRRSignature_DigitalCertificateId","TechnicianSignature_StampNo","MPGType","SignWorkscopeAndCRRSignature_Signature","SignWorkscopeAndCRRSignature_SignedByUserId","SignWorkscopeAndCRRSignature_SignedOn","SignWorkscopeAndCRRSignature_StampNo"], axis = 1)

In [5]:
# Seperation of date and time
df['Dates'] = pd.to_datetime(df['CompletedOn'], format = '%Y%m%d').dt.date
df['Time'] = pd.to_datetime(df['CompletedOn'], format = '%H%m%s').dt.time

# Sorting Values based on Job Order Number
df = df.sort_values(by=['JobOrderNo','Sequence'], ascending = True)
df1 = df1.rename(columns ={'JobOrder':'JobOrderNo'})

In [6]:
#Merging Excel Sheets based on Job Order Number
df3 = pd.merge(df, df1, left_on= 'JobOrderNo', right_on ='JobOrderNo')
df4 = pd.merge(df3,df2, left_on = 'JobOrderNo', right_on = 'JobOrderNo')

In [7]:
# Grouping based on P/N
import matplotlib.pyplot as plt
df4 = df4.sort_values(by = ['JobOrderNo','PartNo','Sequence'])

#Need to remove Null values
df4 = df4.dropna(axis=0, subset=['CompletedOn'])

# Creating lists
PN_dirs = df4['PartNo'].unique().tolist()
PN_list = []
JobOrderList = []
WorkRequiredList = []
WorkStageList = []

# Performing loop to capture every Job
for i in range(round(len(df4)/16)):
   JobOrderNo = df4.iloc[i*16:(i+1)*16:]
   JobOrder = df4.iloc[i*16,0]
   PartNo = df4.iloc[i*16,6]
   WorkRequired = df4.iloc[i*16,9]
   WorkRequiredList.append(WorkRequired)
   JobOrderList.append(JobOrder)
   PN_list.append(PartNo)

# Removing duplicates
New_PN_List = []
New_WorkReqList = [] 
New_JobNumberList= []
for i in PN_list:
    if i not in New_PN_List:
        New_PN_List.append(i)

for i in WorkRequiredList:
  if i not in New_WorkReqList:
    New_WorkReqList.append(i)

for i in JobOrderList:
  if i not in New_JobNumberList:
    New_JobNumberList.append(i)
print ('Hi')

Hi


In [8]:
# Removing Unwanted Data
Work_df = df4.drop(
    ["CompletedOn", "Dates", "Time", "Sequence", "Id", "MPG", "Status", "IncomingPartNo", "OutgoingPartNo"], axis=1)

In [9]:
# Reset Dataframe
Reset_df = Work_df
Average_Total_Costs = []
Costs = ['Labour','Material','Sub Contract','Other','Total Cost']
Cost_Array = []
MinMax_Cost =[]
rows = []

# Creating a loop to calculate mean,min,max values
for a in New_WorkReqList:
    print("Work Performed is:", a)
    print("\n")
    for i in New_PN_List:

        # Calculating Average Costs
        Average_Lab_Cost = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['ActLabCost'].mean()
        Average_Material_Cost = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['ActMatCost'].mean()
        Average_Subc_Cost = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['ActSubcCost'].mean()
        Average_Oth_Cost = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['ActOthCost'].mean()
        Average_Total_Cost = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['ActTotCost'].mean()
        
        # Creating a new dataframe
        rows.append([i,a,Average_Lab_Cost,Average_Material_Cost,Average_Subc_Cost,Average_Oth_Cost,Average_Total_Cost])
        New_Cost_df = pd.DataFrame(rows,columns = ['PartNo','WorkReq','Labour','Materials','SubContract','Others','Total'])
        
        # Creating an array
        Cost_Array = [Average_Lab_Cost,Average_Material_Cost,Average_Subc_Cost,Average_Oth_Cost,Average_Total_Cost]
        LabCost = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['ActLabCost']
        MatCost = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['ActMatCost']
        SubcCost = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['ActSubcCost']
        OthCost = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['ActOthCost']
        TotCost = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['ActTotCost']
        Price_Array = ['LabCost','MatCost','SubcCost','OthCost','TotCost']
        

        # Calculating Total Jobs Found
        Total_Jobs = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['JobOrderNo'].nunique()

        # Calculating Minimum Cos
        Min_Lab_Cost = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['ActLabCost'].min()
        Min_Material_Cost = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['ActMatCost'].min()
        Min_Subc_Cost = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['ActSubcCost'].min()
        Min_Oth_Cost = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['ActOthCost'].min()
        Min_Total_Cost = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['ActTotCost'].min()

        # Calculating Maximum Costs
        Max_Lab_Cost = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['ActLabCost'].max()
        Max_Material_Cost = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['ActMatCost'].max()
        Max_Subc_Cost = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['ActSubcCost'].max()
        Max_Oth_Cost = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['ActOthCost'].max()
        Max_Total_Cost = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['ActTotCost'].max()
        
        # Array
        MinMax_Cost = [Min_Total_Cost,Max_Total_Cost]
        
            

        # Calculating Standard Deviation for Costs
        SD_Lab_Cost = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['ActLabCost'].std()
        if SD_Lab_Cost < 0:
            SD_Lab_Cost = 0
        SD_Lab_Cost = round(SD_Lab_Cost, 2)

        SD_Material_Cost = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['ActMatCost'].std()
        if SD_Material_Cost < 0:
            SD_Material_Cost = 0
        SD_Material_Cost = round(SD_Material_Cost, 2)

        SD_Subc_Cost = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['ActSubcCost'].std()
        if SD_Subc_Cost < 0:
            SD_Subc_Cost = 0
        SD_Subc_Cost = round(SD_Subc_Cost, 2)

        SD_Oth_Cost = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['ActOthCost'].std()
        if SD_Oth_Cost < 0:
            SD_Oth_Cost = 0
        SD_Oth_Cost = round(SD_Oth_Cost, 2)

        SD_Total_Cost = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['ActTotCost'].std()
        if SD_Total_Cost < 0:
            SD_Total_Cost = 0
        SD_Total_Cost = round(SD_Total_Cost, 2)

        # Calculating Average Hours Clocked Per Job
        Average_Hours_Clocked = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['HrClocked'].mean()
        Min_Hours_Clocked = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['HrClocked'].min()
        Max_Hours_Clocked = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['HrClocked'].max()

        # Calculating Standard Deviation
        SD_Hours_Clocked = Work_df.loc[(Work_df['WorkReq'] == a) & (Work_df['PartNo'] == i)]['HrClocked'].std()
        if SD_Hours_Clocked < 0:
            SD_Hours_Clocked = 0.00

        # Rounding values to 2 dp
        Average_Hours_Clocked = round(Average_Hours_Clocked, 2)
        Min_Hours_Clocked = round(Min_Hours_Clocked, 2)
        Max_Hours_Clocked = round(Max_Hours_Clocked, 2)
        SD_Hours_Clocked = round(SD_Hours_Clocked, 2)
        
        # Saving values for the part numbers
        Average_Total_Costs.append(Average_Total_Cost)
        
        # Displaying values
        #print(" The Part Number is:", i)
        #print(" Total Jobs:", Total_Jobs)
        #print("  The average labour cost is:", "$" + str(Average_Lab_Cost))
        #print("  The average Matrial cost is:", "$" + str(Average_Material_Cost))
        #print("  The average SubCon cost is:", "$" + str(Average_Subc_Cost))
        #print("  The average Other cost is:", "$" + str(Average_Oth_Cost))
        #print("  The average Total cost is:", "$" + str(Average_Total_Cost))
        #print("  The average hours clocked is:", str(Average_Hours_Clocked) + " Hours")
        #print("\n")
        #print(str(SD_Hours_Clocked))

        # Resetting dataframe to original
        #Work_df = Reset_df
        #print(Average_Total_Costs)
print(New_Cost_df)
        

Work Performed is: REP


Work Performed is: SST


Work Performed is: O/H


     PartNo WorkReq    Labour  Materials  SubContract  Others    Total
0    505CH2     REP   225.000     100.00       89.500  102.00   516.50
1  ABC81293     REP  1125.625    2201.58     4621.275   90.25  8038.73
2      2758     REP   400.000     300.00      200.000  100.00  1000.00
3    505CH2     SST       NaN        NaN          NaN     NaN      NaN
4  ABC81293     SST       NaN        NaN          NaN     NaN      NaN
5      2758     SST   200.000       0.00        0.000    0.00   200.00
6    505CH2     O/H   245.000     120.00       61.000   72.00   498.00
7  ABC81293     O/H       NaN        NaN          NaN     NaN      NaN
8      2758     O/H       NaN        NaN          NaN     NaN      NaN


In [10]:
# Creating a new dataframe
New_Cost_df = pd.DataFrame(columns = ['PartNo','WorkReq','Labour','Materials','SubContract','Others','Total']) 

print(New_Cost_df)

Empty DataFrame
Columns: [PartNo, WorkReq, Labour, Materials, SubContract, Others, Total]
Index: []


In [12]:
# Importing panel
!pip3 install panel

In [13]:
df1 = pd.read_excel(r"C:\Users\Keithon\Desktop\Capstone\Data\Test.xlsx",sheet_name='JobCost')
Cost_df = df1.drop(['JobOrder','MWC','PartDesc','Strtmlfn','Malfend','Aircraft','HrClocked'], axis = 1)
Total_Part_Number_Avail = list(Cost_df['PartNo'].unique())
Total_Work_Req_Avail = list(Cost_df['WorkReq'].unique())

In [14]:
# Setting up for Time
df1 = pd.read_excel(r"C:\Users\Keithon\Desktop\Capstone\Data\Test.xlsx",sheet_name='JobCost')
Time_df = df1.drop(['JobOrder','SerialNo','MWC','PartDesc','Aircraft','ActTotCost','ActMatCost','ActLabCost','ActSubcCost','ActOthCost'], axis = 1)
New_Rows = []

Time_df['Difference'] = (Time_df['Malfend'] - Time_df['Strtmlfn']).dt.days

# Removing Start and end dates

Time_df = Time_df.drop(['Strtmlfn','Malfend'],axis = 1)
Time_df

,PartNo,WorkReq,HrClocked,Difference
0,505CH2,REP,4.01,36
1,505CH2,REP,5.01,39
2,505CH2,O/H,10.62,46
3,2758,REP,16.20,126
4,2758,SST,5.00,7
5,ABC81293,REP,20.00,44
6,ABC81293,REP,19.70,18
7,ABC81293,REP,22.10,26
8,ABC81293,REP,1.20,0


In [18]:
from matplotlib.figure import Figure
import panel as pn
from panel.interact import interact
import param
import hvplot.pandas
pn.extension()
df1 = pd.read_excel(r"C:\Users\Keithon\Desktop\Capstone\Data\Test.xlsx",sheet_name='JobCost')
Cost_df = df1.drop(['JobOrder','MWC','PartDesc','Strtmlfn','Malfend','Aircraft','HrClocked'], axis = 1)
Cost_df = Cost_df.rename(columns ={'ActTotCost':'Total','ActMatCost':'Materials','ActSubcCost':'SubContract','ActLabCost':'Labour','ActOthCost':'Others'})
Cost_df = Cost_df[['PartNo','WorkReq','Labour', 'Materials', 'SubContract', 'Others', 'Total']]


# Setting up title
Title = '##Interactive Dashboard'

# Setting up panel widgets
Select_X = pn.widgets.Select(name='Part Numbers', options= Total_Part_Number_Avail)
Select_Y = pn.widgets.Select(name='Work Required', options= Total_Work_Req_Avail)
Select_A = pn.widgets.Select(name='Part Numbers', options= Total_Part_Number_Avail)
Select_B = pn.widgets.Select(name='Work Required', options= Total_Work_Req_Avail)

# Setting up plot for box
@pn.depends(Select_X,Select_Y)
def Box_Plot(Select_X,Select_Y):
    Box = Cost_df[(Cost_df['PartNo'] == Select_X) & (Cost_df['WorkReq'] == Select_Y)].hvplot.box(title = 'Part Number is:' + str(Select_X) +'\n'+'Work Performed is:'+str(Select_Y) ,value_label='Costs ($)',xlabel = 'Different Types of Cost',legend = False, width = 1100, height = 700)
    return Box


# Setting up Mahour tracking
@pn.depends(Select_A,Select_B)
def Hour_Plot(Select_A,Select_B):
    Time_Check = Time_df[(Time_df['PartNo'] == Select_A) & (Time_df['WorkReq'] == Select_B)].hvplot.box(title = 'Part Number is:' + str(Select_A) +'\n'+'Work Performed is:'+str(Select_B) ,y = 'Difference',ylabel = 'Number of Days',xlabel='Average Turn Around Days',legend = False,width = 500, height = 700)
    return Time_Check

@pn.depends(Select_A,Select_B)
def Day_Plot(Select_A,Select_B):
    ManHour = Time_df[(Time_df['PartNo'] == Select_A) & (Time_df['WorkReq'] == Select_B)].hvplot.box(title = 'Part Number is: '+ str(Select_A)+ '\n' + 'Work Hours',y = 'HrClocked',ylabel = 'Hours',xlabel='ManHour',legend = False, width = 500, height = 700)
    return ManHour
    
# For Manhour tab
p2 = pn.Row(
        pn.WidgetBox(Title,Select_A,Select_B),
        pn.Column(Hour_Plot),
        pn.bind(Day_Plot,Select_A,Select_B), name = 'ManHour Tracking')

# For Cost tab
p1 = pn.Row(
        pn.WidgetBox(Title,Select_X,Select_Y),
        pn.bind(Box_Plot,Select_X,Select_Y), name = 'Costs')


tabs = pn.Tabs(p1,p2, dynamic = True)
tabs.servable()
tabs.show()


Launching server at http://localhost:54822


In [38]:
#Creating server
!pip install virtualenv
!pip install flask
from flask import Flask
!pip install heroku

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\dateutil\\easter.py'
Consider using the `--user` option or check the permissions.




  Created wheel for heroku: filename=heroku-0.1.4-py3-none-any.whl size=12364 sha256=3166887ebfe0da466c6f47671d638b88167ef0c6b3f8945d22ab90a1a939a550
  Stored in directory: c:\users\keithon\appdata\local\pip\cache\wheels\61\7f\9a\af879a4e9e8d6ff17b1cdfe66336fbc86ea9e30707097515b9
  Created wheel for python-dateutil: filename=python_dateutil-1.5-py3-none-any.whl size=203004 sha256=545d3f13be4834eae4fd11481f89cd80ae3362c7e885002c1651ff6f978da0af
  Stored in directory: c:\users\keithon\appdata\local\pip\cache\wheels\7a\63\dd\75ae41c09b61a072b15653bfc4abff3d3bfb997c93338bcf55
Successfully built heroku python-dateutil
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.2
    Uninstalling python-dateutil-2.8.2:


SyntaxError: invalid syntax (2021433499.py, line 1)